# SLIM with BPR Recommender

In [ ]:
def init_kaggle(run_on_kaggle, secret_name):
    if run_on_kaggle:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        personal_token = user_secrets.get_secret(secret_name)
        
        import subprocess
        subprocess.run(["git", "clone", "https://" + personal_token + "@github.com/alecontuIT/rec_sys_challenge2022.git"])
        
        import os
        os.chdir("./rec_sys_challenge2022")
        
        subprocess.run(["pip", "install", "-r", "requirements.txt"])
        subprocess.run(["python", "run_compile_all_cython.py"])
        return True
    
    else:
        return False

In [ ]:
kaggle = init_kaggle(False, "recsys_git_token")

In [ ]:
import utils
from recmodels import SLIM_BPRRec
from scipy.stats import loguniform
from Evaluation.Evaluator import EvaluatorHoldout
from skopt.space import Real, Integer, Categorical
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

In [ ]:
recommender_class = SLIM_BPRRec
dataset_version = "interactions-all-ones"

In [ ]:
URM_all, URM_train, URM_val, ICM = utils.get_data_global_sample(dataset_version=dataset_version, 
                                                                train_percentage=0.7,
                                                                setSeed=True)

In [ ]:
evaluator_validation = EvaluatorHoldout(URM_val, cutoff_list=[10])

## Hyperparameter search

In [ ]:
free_mem_threshold = 0.1

In [ ]:
#'''
hyperparameters_range_dictionary = {
    "topK": Integer(1, 1000),
    "epochs": Categorical([1500]),
    "symmetric": Categorical([True, False]),
    "sgd_mode": Categorical(['adagrad', 'rmsprop', 'adam', 'sgd']),
    "lambda_i": Real(low = 1e-5, high = 1e3, prior = 'log-uniform'),
    "lambda_j": Real(low = 1e-5, high = 1e3, prior = 'log-uniform'),
    "learning_rate": Real(low = 1e-5, high = 1, prior = 'log-uniform')
}

'''
hyperparameters_range_dictionary = {
    "topK" : Integer(1, 1000),
    "epochs": Categorical([300]),
    "lambda_i" : Real(low= 1e-5, high=1e3, prior="log-uniform"),
    "lambda_j" : Real(low= 1e-5, high=1e3, prior="log-uniform"),
    "learning_rate" : Real(low= 1e-5, high=1e-1, prior="log-uniform"),
    "symmetric": Categorical([True, False]),
    "sgd_mode": Categorical(['adagrad', 'rmsprop', 'adam', 'sgd']),
    #"gamma": Categorical([0.995]),
    #"beta_1": Categorical([0.9]), 
    #"beta_2": Categorical([0.999])
}
'''

In [ ]:
earlystopping_keywargs = {
    "validation_every_n": 5,
    "stop_on_validation": True,
    "evaluator_object": evaluator_validation,
    "lower_validations_allowed": 5,
    "validation_metric": "MAP",
}

In [ ]:
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train, free_mem_threshold],     
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {"positive_threshold_BPR": None,
                        'train_with_sparse_weights': False,
                        'allow_train_with_sparse_weights': False},
    EARLYSTOPPING_KEYWORD_ARGS = earlystopping_keywargs, 
)

In [ ]:
utils.bayesian_search(
    recommender_class, 
    recommender_input_args, 
    hyperparameters_range_dictionary, 
    evaluator_validation,
    dataset_version=dataset_version,
    n_cases=150,
    perc_random_starts=0.1,
    resume_from_saved = True
)

In [ ]:
import os
import zipfile
current_temp_folder = utils.get_hyperparams_search_output_folder(recommender_class, dataset_version) + "SLIM_BPR_Recommender_metadata"
print(current_temp_folder)
with zipfile.ZipFile(current_temp_folder + ".zip", 'w', compression=zipfile.ZIP_DEFLATED) as myzip:
    for file_to_compress in os.listdir(current_temp_folder):
        print(file_to_compress)
        myzip.write(current_temp_folder + "/" + file_to_compress, arcname = file_to_compress)

## Best Model

In [ ]:
recommender = utils.fit_best_recommender(recommender_class, URM_all, dataset_version)
utils.submission(recommender, dataset_version)